*** UNDER CONSTRUCTION ***


# HTLC-Timeout and HTLC-Success Transactions
## Prerequisite knowledge
### For all notebooks
- A high level understanding of the bitcoin e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook), in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
- A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
- [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).
- A high level understanding of the lightning e.g. [Mastering Lightning Network](https://github.com/lnbook/lnbook), in particular [Chapter7](https://github.com/lnbook/lnbook/blob/develop/07_payment_channels.asciidoc), [Chapter 8](https://github.com/lnbook/lnbook/blob/develop/08_routing_htlcs.asciidoc) and [Chapter 9](https://github.com/lnbook/lnbook/blob/develop/09_channel_operation.asciidoc).

### Specific to this notebook
- SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
- Bech32 addresses - '[Addresses chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
- Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
- Lightning Network BOLT #2: '[Peer Protocol for Channel Management](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1)'
- Lightning Network BOLT #3: '[Commitment Transactions](https://github.com/lightning/bolts/blob/master/03-transactions.md#commitment-transaction)'
- Finite Fields, Elliptic Curves and Serialization e.g. [Programming Bitcoin](https://github.com/jimmysong/programmingbitcoin), in particular [Chapter 1](https://github.com/jimmysong/programmingbitcoin/blob/master/ch01.asciidoc), [Chapter2](https://github.com/jimmysong/programmingbitcoin/blob/master/ch02.asciidoc), [Chapter 3](https://github.com/jimmysong/programmingbitcoin/blob/master/ch03.asciidoc) and [Chapter 4](https://github.com/jimmysong/programmingbitcoin/blob/master/ch04.asciidoc).

## HTLC-Timeout and HTLC-Success Transactions

The HTLC-Timeout and HTLC-Success Transactions are almost identical, except the HTLC-timeout transaction is timelocked. Both HTLC-timeout/HTLC-success transactions can be spent by a valid penalty transaction.

* version: 2
* locktime: 0 for HTLC-success, cltv_expiry for HTLC-timeout
* txin count: 1
    * txin[0] outpoint: txid of the commitment transaction and output_index of the matching HTLC output for the HTLC transaction
    * txin[0] sequence: 0 (set to 1 for option_anchors)
    * txin[0] script bytes: 0
    * txin[0] witness stack: 0 <remotehtlcsig> <localhtlcsig>  <payment_preimage> for HTLC-success, 0 <remotehtlcsig> <localhtlcsig> <> for HTLC-timeout

* txout count: 1
    * txout[0] amount: the HTLC amount_msat divided by 1000 (rounding down) minus fees in satoshis (see Fee Calculation)
    * txout[0] script: version-0 P2WSH with witness script as shown below

if option_anchors applies to this commitment transaction, SIGHASH_SINGLE|SIGHASH_ANYONECANPAY is used as described in [BOLT #5](https://github.com/lightning/bolts/blob/master/05-onchain.md#generation-of-htlc-transactions).

The witness script for the output is:

```
OP_IF
    # Penalty transaction
    <revocationpubkey>
OP_ELSE
    `to_self_delay`
    OP_CHECKSEQUENCEVERIFY
    OP_DROP
    <local_delayedpubkey>
OP_ENDIF
OP_CHECKSIG
```


## Setup
### Requirements

For this exercise we'll run the previous notebook to create the commitment transactions beetween with offer and received HTLC.

**You'll need to edit these next line for your local setup.**

In [1]:
# run notebook
%run "/home/pins-dev/Projects/lightning-tx-tutorial/Chapter 4 - Unilateral Closing Transaction/Unilateral Closing Transaction.ipynb"

Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Revocation Basepoint Private Key: c17ac3952ca414190074d1e59ea03fbae253196173908dc8b131af6bd2cc8161
Alice Revocation Basepoint Public Key: 03649c4f865bec74b0a186deef4defad51cfdc141443e38074ea05a7835a953a49
Alice HTLC Basepoint Private Key: 763ae49a20e6668c88602c782716dd83ba6c4cc0333b38810e2bcd7b22c871ac
Alice HTLC Basepoint Public Key: 02816fde4150e4dfcac94eff0b821448fb70f57a56148ba2206cd9b2fd0cc20bdf
Alice Payment Basepoint Private Key: 72d8c12971b58076a1f27eb7938ca442f0b210762b23637443ac2e99dac352a6
Alice Payment Basepoint Public Key: 025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Alice Delayed Payment Basepoint Private Key: 7cafce00c54e7241894dcc7c3beaca29dd354139fdb6182198d6c5f1063bfe8d
Alice Delayed Payment Basepoint Public Key: 034aa35219136bb238e072341b20a4bf8fb44a83cdb73dd2bd9

### Creating the HTLC-Timeout and HTLC-Success Transactions

Now that the commitment transaction, containing HTLC outputs, was published forcing close the channel, we can create the HTLC-Timeout and/or HTLC-Success Transactions to reclaim the funds locked on HTLCs.

Let's start with simulating an HTLC Timeout transaction.

In [ ]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER
marker = bytes.fromhex("00")

# FLAG
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# OUTPUT 1 to Alice Change Address
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output1_spk' 
output1_spk = bytes.fromhex("0014") + pk_hash


# OUTPUT 2 to Channel Funding Address
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output2_spk'
output2_spk = bytes.fromhex("0020") + script_hash

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_channel_funding_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)

print("unsigned_channel_funding_tx: ", unsigned_channel_funding_tx.hex())

